# MeterReader - The Trainer

This notebok implements the YOLOv11 based trainers for the models used by the meterreader

- Frames : Get Extract the Frame inside of the meter from the original image
- Counter : Extract the Counter from within the frame
- Digits: Get the value of each digit of the Counter

## Training Data

The training data is taken from the respective Roboflow projects. The data is annotated using either Roboflow or manually via script (Digits)

## Models

The resulting models are saved in the same directory as the downloaded roboflow datasets 





## Citations and Acknowledgements

@software{yolo11_ultralytics,
  author = {Glenn Jocher and Jing Qiu},
  title = {Ultralytics YOLO11},
  version = {11.0.0},
  year = {2024},
  url = {https://github.com/ultralytics/ultralytics},
  orcid = {0000-0001-5950-6979, 0000-0002-7603-6750, 0000-0003-3783-7069},
  license = {AGPL-3.0}
}

In [ ]:
# Uses the Ultralytics YOLOv11 Libraries
import ultralytics
ultralytics.checks()

from ultralytics import YOLO
import torch

# Datasets are stored, annotated, on roboflow
from roboflow import Roboflow

# Load custom configurations. Variables accessible with config.get('<<VARIABLE_NAME>>')
from config import ConfigLoader

import os, shutil

In [ ]:
# Load the config variables
config = ConfigLoader()

print("The loaded configuration:")

config.print_config()

# Check to see if running on a Mac or not (Apple Silicon)
device_to_use = config.get('DEVICE')

if torch.backends.mps.is_available():
    device_to_use = 'mps'

print(f"Device used for training: {device_to_use}")


In [ ]:
# Load the roboflow datasets

rf = Roboflow(api_key=config.get('RF_API_KEY'))
workspace = rf.workspace(config.get('RF_WORKSPACE_NAME'))
#project = workspace.project(config.get('RF_PROJECT_NAME_DIGITS'))
# project = workspace.project(config.get('RF_PROJECT_NAME_FRAME'))
project = workspace.project(config.get('RF_PROJECT_NAME_COUNTERS'))

# version = project.version(config.get('RF_VERSION_DIGITS'))
# version = project.version(config.get('RF_VERSION_FRAME'))
version = project.version(config.get('RF_VERSION_COUNTERS'))


dataset = version.download("yolov11")
# dataset = version.download("yolov11", location="../datasets")
print(f"Dataset Location: {dataset.location}")




In [ ]:
# Train

# Load a model
model = YOLO("yolo11n.pt")  # load a pretrained model (recommended for training)

run_name=os.path.basename(dataset.location)
project_name="meterreader_YOLO"

# Train the model with MPS
results = model.train(data=f"{dataset.location}/data.yaml", cache=True, epochs=50, imgsz=[160,160], device="mps", 
                                                            name=run_name, project=project_name, exist_ok=True)

resulting_model = f"{results.save_dir}/weights/best.pt"

In [ ]:
# Export the trained custom model

if results:
    trained_model = YOLO(resulting_model)
    trained_model.export(device='cpu',  imgsz=[192,800],
                 int8=True, data=f"{dataset.location}/data.yaml",
                 name=run_name, project=project_name, exist_ok=True)


In [ ]:

# Load a model
print(f"Loading the model for testing from {resulting_model}")
digits_model = YOLO(resulting_model)  # load prev. trained model

images = f"/Users/yonz/Workspace/images/{run_name}/*.jpg"

# run prediction on some test images
results = digits_model(images, device='mps', imgsz=[192,800], conf=0.4, iou=0.5,
                        save=True, save_crop =True, save_txt=True,
                        name=f"{run_name}/", project=f"{project_name}", exist_ok=True)


print(f"Done. Processed {len(results)} images")